In [12]:
from transformers import BertTokenizer,TFBertForSequenceClassification
import re
from sklearn.preprocessing import LabelEncoder
from tensorflow.nn import softmax
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import BertTokenizer
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical

np.random.seed(42)


# Dynamically allocate GPU memory
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
from transformers import BertTokenizer,TFBertForSequenceClassification
import re
from sklearn.preprocessing import LabelEncoder
from tensorflow.nn import softmax
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import BertTokenizer
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical

np.random.seed(42)


# Dynamically allocate GPU memory
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
label_encoder = LabelEncoder()
num_classes = 111
data = []
data_set = "latest_class_10000.csv"
model = TFBertForSequenceClassification.from_pretrained('model_100000_07_14.h5', num_labels=num_classes)
with open(data_set, 'r') as csvfile:
        for line in csvfile:
            if len(line) > 5:
                a, b = line.split('",', 1)
                a, b = a[2:], b[2:-2]
                bs = re.findall(r"'(.*?)'", b, re.DOTALL)
                bs = ", ".join(bs)
                data.append([a,bs])
# data = data[:10]
# Separate code from actions
code = [item[0] for item in data]
actions = [item[1] for item in data]
df = pd.DataFrame(list(zip(code, actions)), columns=['code', 'actions'])
label_encoder.fit(df['actions'])
actions_encoded = label_encoder.transform(df['actions'])
action_one_hot = to_categorical(actions_encoded, num_classes = len(set(df['actions'])))
action_argmax = np.argmax(action_one_hot)
c = code[0]
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
first_input_tokenized = tokenizer.encode(c, add_special_tokens=True, max_length=256, truncation=True, padding="max_length")
# ind_action_encode = label_encoder.transform(a)
# ind_action_one_hot = to_categorical(ind_action_encode, num_classes = num_classes)
# ind_act_argmax = np.argmax(ind_action_one_hot)
# Make predictions using the model
logits = model.predict([first_input_tokenized])

# Extract the logits from the TFSequenceClassifierOutput object
logits_values = logits.logits

# Convert logits to probabilities
probs = softmax(logits_values)
top_5_indices = np.argsort(probs, axis=1)[:, -5:]
# Decode the classes
predicted_top_5_labels = [label_encoder.inverse_transform(indices) for indices in top_5_indices]
predicted_top_5_labels = [list(indices) for indices in predicted_top_5_labels][0]
print(predicted_top_5_labels)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at model_100000_07_14.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.
2023-08-21 15:51:10.578942: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 4s 4s/step
['map(), take()', 'collect()', 'groupBy()', 'map(), reduce()', 'map()']


In [23]:
# Tokenize the code
input_ids = [tokenizer.encode(c, add_special_tokens=True) for c in df['code']]

# Pad sequences to ensure uniform length
code_padded = pad_sequences(input_ids, maxlen=256, dtype="long", value=0, truncating="post", padding="post")

In [24]:
assert tokenizer.decode(first_input_tokenized) == tokenizer.decode(code_padded[0])
assert label_encoder.inverse_transform([action_argmax]) == label_encoder.inverse_transform([ind_act_argmax])

In [20]:
# Make predictions using the model
logits = model.predict([first_input_tokenized])

# Extract the logits from the TFSequenceClassifierOutput object
logits_values = logits.logits

# Convert logits to probabilities
probs = softmax(logits_values)

# Decode the classes
predicted_top_5_labels = [label_encoder.inverse_transform(indices) for indices in top_5_indices]
predicted_top_5_labels = [list(indices) for indices in predicted_top_5_labels][0]
print(predicted_top_5_labels)

1/1 [==============================] - 0s 166ms/step
['collect()', 'groupBy()', 'map(), reduce()', 'map()']


In [1]:
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.nn import softmax
from transformers import BertTokenizer, TFBertForSequenceClassification


def initialize_gpu():
    """
    Dynamically allocate GPU memory.
    """
    physical_devices = tf.config.list_physical_devices('GPU')
    if physical_devices:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)


def load_dataset(filepath):
    """
    Load the dataset from the given CSV file.
    
    Args:
    - filepath (str): Path to the CSV file.
    
    Returns:
    - List[Tuple[str, str]]: List of (code, action) pairs.
    """
    data = []
    with open(filepath, 'r') as csvfile:
        for line in csvfile:
            if len(line) > 5:
                a, b = line.split('",', 1)
                a, b = a[2:], b[2:-2]
                bs = re.findall(r"'(.*?)'", b, re.DOTALL)
                bs = ", ".join(bs)
                data.append((a,bs))
    return data


def preprocess_data(data, num_classes):
    """
    Preprocess the data.
    
    Args:
    - data (List[Tuple[str, str]]): List of (code, action) pairs.
    - num_classes (int): Number of action classes.
    
    Returns:
    - DataFrame: Pandas DataFrame with code and actions columns.
    - ndarray: One-hot encoded actions.
    """
    # Separate code from actions
    code = [item[0] for item in data]
    actions = [item[1] for item in data]
    
    df = pd.DataFrame(list(zip(code, actions)), columns=['code', 'actions'])
    label_encoder.fit(df['actions'])
    actions_encoded = label_encoder.transform(df['actions'])
    action_one_hot = to_categorical(actions_encoded, num_classes=len(set(df['actions'])))
    
    return df, action_one_hot


def predict_top_5_labels(model, tokenized_input, label_encoder):
    """
    Predict the top 5 labels for a tokenized input.
    
    Args:
    - model (TFBertForSequenceClassification): The BERT model.
    - tokenized_input (List[int]): Tokenized input sequence.
    - label_encoder (LabelEncoder): Encoder for action labels.
    
    Returns:
    - List[str]: Top 5 predicted labels.
    """
    logits = model.predict([tokenized_input])
    logits_values = logits.logits
    probs = softmax(logits_values)
    
    top_5_indices = np.argsort(probs, axis=1)[:, -5:]
    predicted_top_5_labels = [label_encoder.inverse_transform(indices) for indices in top_5_indices]
    return [list(indices) for indices in predicted_top_5_labels][0]


# Initialization
np.random.seed(42)
initialize_gpu()
label_encoder = LabelEncoder()
num_classes = 111

# Load the model and data
model = TFBertForSequenceClassification.from_pretrained('model_100000_07_14.h5', num_labels=num_classes)
data = load_dataset("latest_class_10000.csv")

# Preprocess the data
df, action_one_hot = preprocess_data(data, num_classes)

# Tokenize a sample input
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
first_input_tokenized = tokenizer.encode(df['code'][0], add_special_tokens=True, max_length=256, truncation=True, padding="max_length")

# Predict the top 5 labels
top_5_labels = predict_top_5_labels(model, first_input_tokenized, label_encoder)
print(top_5_labels)


Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2023-08-21 15:43:29.763831: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-08-21 15:43:29.763961: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
Some layers from the model checkpoint at model_100000_07_14.h5 were not used when initializing TFBertForSequenceClassification: ['dropout_75']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkp

1/1 [==============================] - 5s 5s/step
['map(), take()', 'collect()', 'groupBy()', 'map(), reduce()', 'map()']


In [16]:
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.nn import softmax
from transformers import BertTokenizer, TFBertForSequenceClassification


class BertClassifier:
    def __init__(self, model_path, num_classes):
        self.model = TFBertForSequenceClassification.from_pretrained(model_path, num_labels=num_classes)
        self.label_encoder = LabelEncoder()
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.initialize_gpu()

    @staticmethod
    def initialize_gpu():
        """Dynamically allocate GPU memory."""
        physical_devices = tf.config.list_physical_devices('GPU')
        if physical_devices:
            tf.config.experimental.set_memory_growth(physical_devices[0], True)

    @staticmethod
    def load_dataset(filepath):
        """Load the dataset from the given CSV file."""
        data = []
        with open(filepath, 'r') as csvfile:
            for line in csvfile:
                if len(line) > 5:
                    a, b = line.split('",', 1)
                    a, b = a[2:], b[2:-2]
                    bs = re.findall(r"'(.*?)'", b, re.DOTALL)
                    bs = ", ".join(bs)
                    data.append((a, bs))
        return data

    def preprocess_data(self, data, num_classes):
        """Preprocess the data."""
        code = [item[0] for item in data]
        actions = [item[1] for item in data]

        df = pd.DataFrame(list(zip(code, actions)), columns=['code', 'actions'])
        return df

    def fit_label(self, df):
        self.label_encoder.fit(df['actions'])

    def predict_top_5_labels(self, code):
        """Predict the top 5 labels for a given code."""
        tokenized_input = self.tokenizer.encode(code, add_special_tokens=True, max_length=256, truncation=True, padding="max_length")
        logits = self.model.predict([tokenized_input])
        logits_values = logits.logits
        probs = softmax(logits_values)

        top_5_indices = np.argsort(probs, axis=1)[:, -5:]
        predicted_top_5_labels = [self.label_encoder.inverse_transform(indices) for indices in top_5_indices]
        return [list(indices) for indices in predicted_top_5_labels][0]

    def predict_top_class(self,code):
        tokenized_input = self.tokenizer.encode(code, add_special_tokens=True, max_length=256, truncation=True, padding="max_length")
        logits = self.model.predict([tokenized_input])
        logits_values = logits.logits
        probs = softmax(logits_values)
        # Find the class with the maximum probability
        predicted_classes = np.argmax(probs, axis=1)
        predicted_classes = self.label_encoder.inverse_transform(predicted_classes)
        return predicted_classes[0]




class Top5Predictions:
    def __init__(self):
        self.classifier = BertClassifier('model_100000_07_14.h5', 111)
        data = self.classifier.load_dataset("latest_class_10000.csv")
        df = self.classifier.preprocess_data(data, 111)
        self.classifier.fit_label(df)
    
    def make_prediction(self, code):
        return self.classifier.predict_top_5_labels(code)

top5 = Top5Predictions()
code_snap = "for num in numbers:\
count += 1"
top_5_labels = top5.make_prediction(code_snap)
top_class = top5.classifier.predict_top_class(code_snap)
print(top_5_labels)
print(top_class)


Some layers from the model checkpoint at model_100000_07_14.h5 were not used when initializing TFBertForSequenceClassification: ['dropout_75']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at model_100000_07_14.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.
2023-08-22 09:46:02.495954: I tensorflow/core/grappler/optimizers/c

1/1 [==============================] - 1s 560ms/step
['sortBy()', 'map(), count()', 'sum()', 'collect()', 'count()']
count()


In [9]:
top5 = Top5Predictions()
code_snap = "for i in range(len(data0)):\n\tdata[i] *= 3"
top_5_labels = top5.make_prediction(code_snap)
print(top_5_labels)

Some layers from the model checkpoint at model_100000_07_14.h5 were not used when initializing TFBertForSequenceClassification: ['dropout_75']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at model_100000_07_14.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.
2023-08-21 16:06:31.510288: I tensorflow/core/grappler/optimizers/c

1/1 [==============================] - 4s 4s/step
['map(), count()', 'map(), reduce()', 'groupBy()', 'collect()', 'map()']
